Stand: 26.03.25

env: konstanz_karte

## Interaktive Karte

Verbinden aller bisheriger geografischen Infos: 
- Messstationen
    - DWD - Konstanz
    - Daten Schweiz
    - 12 Sensoren Konstanz
        - messstationen_meta.csv (von mir erstellt aus den Excel Dateien der Sonsordaten von Konstanz auf der Webseite)
- LCZ ??
- shape file Konstanz mit Stadtteilen
    - "Kleinräumige_Gliederung_4330766187849128049.geojson"
- Maites Pizza Infos

In [1]:
import geopandas as gpd
import geoviews as gv
import geoviews.tile_sources as gts
import holoviews as hv
import hvplot.pandas
import pandas as pd
import panel as pn

## 1) Alle Wichtigen Daten einlesen

In [2]:
# Karte von Konstanz mit Grenzen der Stadtteile + Stadtteilnamen
konstanz_karte = gpd.read_file("maja_geodaten/Kleinräumige_Gliederung_4330766187849128049.geojson")

# Punkte der Sensorstandorte
konstanz_sensoren_pkt = pd.read_csv("maja_geodaten/messstationen_meta.csv", sep=";")

# Polygone von Stadtklimaexperte Tim - selbst gemalt von ihm und klassifiziert - Grundlage für die den LCZ Generator
# das sind alle lcz die Tim verwendet hat
layers_to_use = ['2', '5', '6', '8', '9', 'A', 'B', 'C', 'D', 'E', 'G']
# 5 und 6 und C und D haben ja die gleiche Farbe
lcz_colors = ['#D10000', '#FF6600', '#FF6600', '#B3B3B3', '#FFCCAA',
              '#006A00', '#00AA00', '#B9DB79', '#B9DB79', '#000000', '#6A6AFF']

frames = []
for layer in layers_to_use:
    gdf = gpd.read_file("maja_geodaten/4381f9078b2704eac4c9ab8c1631439c4ccff939.kml",
                        driver="KML", layer=layer)
    gdf["tims_layer"] = layer  # Layerkennung hinzufügen
    frames.append(gdf)

# Alles in ein GeoDataFrame zusammenführen
tims_gdf = pd.concat(frames).reset_index(drop=True)
tims_gdf.dropna(axis=1, how='all', inplace=True)

# Farbcodes je Layer zuweisen
color_dict = dict(zip(layers_to_use, lcz_colors))
tims_gdf['color'] = tims_gdf['tims_layer'].map(color_dict)

# tims_gdf = pd.concat(frames).reset_index(drop=True)
# tims_gdf.dropna(axis=1, how='all', inplace=True)
# color_dict = dict(zip(tims_gdf['tims_layer'].unique(), lcz_colors))
# tims_gdf['color'] = tims_gdf['tims_layer'].map(color_dict)

# Testen ob alles da ist
display("Konstanz Karte:", konstanz_karte.head())
display("Konstanzer Sensoren Punkte:", konstanz_sensoren_pkt.head())
# display("Tims LCZ Polygone:", tims_gdf.head())
print("Man könnte Tims Layer noch säubern.")
# print(tims_gdf["tims_layer"].unique())
# print(tims_gdf["Name"].unique())

# print(tims_gdf['color'].unique())
# tims_gdf.hvplot(geo=True, color=tims_gdf['color'], legend=False, title="LCZ-Zonen in Konstanz")

'Konstanz Karte:'

,OBJECTID,STT_NR,STT,STT_NAME,AREA,LEN,geometry
0,1043,090,90,Industriegebiet,None,None,"MULTIPOLYGON (((9.12893 47.68435, 9.12914 47.6..."
1,1044,100,100,Egg,None,None,"POLYGON ((9.18497 47.70226, 9.18523 47.70216, ..."
2,1045,110,110,Litzelstetten,None,None,"MULTIPOLYGON (((9.17016 47.6981, 9.17116 47.69..."
3,1046,120,120,Dingelsdorf,None,None,"POLYGON ((9.17454 47.73315, 9.17453 47.73307, ..."
4,1047,130,130,Dettingen,None,None,"POLYGON ((9.10776 47.76265, 9.10799 47.76259, ..."


'Konstanzer Sensoren Punkte:'

,name,Geogr.Laenge,Geogr.Breite
0,Döbele,9.168898,47.658239
1,Europapark,9.162823,47.667215
2,Fähre Staad,9.209338,47.682119
3,Friedrichstrasse,9.183755,47.675575
4,Herose-Park,9.174574,47.669055


Man könnte Tims Layer noch säubern.


## 2) Karte erstellen

In [9]:

plots = []
for layer, farbe in zip(layers_to_use, lcz_colors):
    gdf_layer = tims_gdf[tims_gdf['tims_layer'] == layer]
    p = gdf_layer.hvplot(geo=True, color=farbe, line_color="black", legend=True, alpha=0.5)
    plots.append(p)

final_lcz_overlay = hv.Overlay(plots).opts(
    title="LCZ-Zonen in Konstanz – Tims Polygone",
    width=900,
    height=600,
    active_tools=['wheel_zoom', 'pan']
)

# -------------------------------
# 🗺️ 3. OpenStreetMap als Basiskarte
# -------------------------------
osm_tile = gts.OSM.opts(width=900, height=600)

# -------------------------------
# 🌍 4. Karte kombinieren: OSM + LCZ
# -------------------------------
map_with_osm = osm_tile * final_lcz_overlay
map_with_osm

:Overlay
   .WMTS.I        :WMTS   [Longitude,Latitude]
   .Polygons.I    :Polygons   [Longitude,Latitude]
   .Polygons.II   :Polygons   [Longitude,Latitude]
   .Polygons.III  :Polygons   [Longitude,Latitude]
   .Polygons.IV   :Polygons   [Longitude,Latitude]
   .Polygons.V    :Polygons   [Longitude,Latitude]
   .Polygons.VI   :Polygons   [Longitude,Latitude]
   .Polygons.VII  :Polygons   [Longitude,Latitude]
   .Polygons.VIII :Polygons   [Longitude,Latitude]
   .Polygons.IX   :Polygons   [Longitude,Latitude]
   .Polygons.X    :Polygons   [Longitude,Latitude]
   .Polygons.XI   :Polygons   [Longitude,Latitude]